In [1]:
!pip install scikit-learn


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import shutil

# Define paths
image_folder = r"D:\Belajar ML\training\image_2"
label_folder = r"D:\Belajar ML\training\label_2"
train_image_folder = "dataset/train/images"
val_image_folder = "dataset/val/images"
train_label_folder = "dataset/train/labels"
val_label_folder = "dataset/val/labels"

# Define the mapping of class names to IDs
class_name_to_id = {
    "Car": 0, 
    "Pedestrian": 1, 
    "Van": 2, 
    "Cyclist": 3, 
    "Truck": 4, 
    "Misc": 5, 
    "Tram": 6, 
    "Person_sitting": 7, 
    "DontCare": 8
}

# Image dimensions (adjust based on your dataset)
img_width = 1242  # Example width
img_height = 375  # Example height

# Create necessary folders
os.makedirs(train_image_folder, exist_ok=True)
os.makedirs(val_image_folder, exist_ok=True)
os.makedirs(train_label_folder, exist_ok=True)
os.makedirs(val_label_folder, exist_ok=True)

print("Paths and class mappings set up successfully!")


Paths and class mappings set up successfully!


In [3]:
def convert_kitti_to_yolo(kitti_file, output_dir, class_mapping, img_width, img_height):
    """
    Converts a single KITTI label file to YOLOv5 format and saves it in the specified output directory.
    """
    with open(kitti_file, 'r') as f:
        lines = f.readlines()

    yolo_lines = []
    for line in lines:
        data = line.split()
        label = data[0]
        if label == "DontCare":  # Skip DontCare labels
            continue
        
        # Extract bounding box coordinates
        xmin, ymin, xmax, ymax = map(float, data[4:8])
        
        # Convert to YOLO format
        center_x = (xmin + xmax) / 2 / img_width
        center_y = (ymin + ymax) / 2 / img_height
        width = (xmax - xmin) / img_width
        height = (ymax - ymin) / img_height
        
        # Map class to index
        class_id = class_mapping.get(label, -1)
        if class_id == -1:
            continue
        
        yolo_lines.append(f"{class_id} {center_x:.6f} {center_y:.6f} {width:.6f} {height:.6f}")

    # Write to YOLO format file
    output_file = os.path.join(output_dir, os.path.basename(kitti_file))
    with open(output_file, 'w') as f:
        f.write("\n".join(yolo_lines))
    
    print(f"Converted: {kitti_file} -> {output_file}")


In [4]:
# Get all image files and corresponding labels
image_files = sorted(os.listdir(image_folder))
label_files = sorted(os.listdir(label_folder))

# Limit to 500 training and 200 validation images
max_train = 500
max_val = 200

train_images = image_files[:max_train]
val_images = image_files[max_train:max_train + max_val]

print(f"Train images: {len(train_images)} (max: {max_train})")
print(f"Validation images: {len(val_images)} (max: {max_val})")


Train images: 500 (max: 500)
Validation images: 200 (max: 200)


In [5]:
# Process training data (limited to 500 images)
for img_file in train_images:
    src_img_path = os.path.join(image_folder, img_file)
    dst_img_path = os.path.join(train_image_folder, img_file)
    shutil.copy(src_img_path, dst_img_path)

    label_file = img_file.replace(".png", ".txt")
    src_label_path = os.path.join(label_folder, label_file)
    dst_label_path = os.path.join(train_label_folder, label_file)
    
    if os.path.exists(src_label_path):
        convert_kitti_to_yolo(src_label_path, train_label_folder, class_name_to_id, img_width, img_height)

# Process validation data (limited to 200 images)
for img_file in val_images:
    src_img_path = os.path.join(image_folder, img_file)
    dst_img_path = os.path.join(val_image_folder, img_file)
    shutil.copy(src_img_path, dst_img_path)

    label_file = img_file.replace(".png", ".txt")
    src_label_path = os.path.join(label_folder, label_file)
    dst_label_path = os.path.join(val_label_folder, label_file)
    
    if os.path.exists(src_label_path):
        convert_kitti_to_yolo(src_label_path, val_label_folder, class_name_to_id, img_width, img_height)

print("Dataset split and conversion completed (500 training, 200 validation)!")


Converted: D:\Belajar ML\training\label_2\000000.txt -> dataset/train/labels\000000.txt
Converted: D:\Belajar ML\training\label_2\000001.txt -> dataset/train/labels\000001.txt
Converted: D:\Belajar ML\training\label_2\000002.txt -> dataset/train/labels\000002.txt
Converted: D:\Belajar ML\training\label_2\000003.txt -> dataset/train/labels\000003.txt
Converted: D:\Belajar ML\training\label_2\000004.txt -> dataset/train/labels\000004.txt
Converted: D:\Belajar ML\training\label_2\000005.txt -> dataset/train/labels\000005.txt
Converted: D:\Belajar ML\training\label_2\000006.txt -> dataset/train/labels\000006.txt
Converted: D:\Belajar ML\training\label_2\000007.txt -> dataset/train/labels\000007.txt
Converted: D:\Belajar ML\training\label_2\000008.txt -> dataset/train/labels\000008.txt
Converted: D:\Belajar ML\training\label_2\000009.txt -> dataset/train/labels\000009.txt
Converted: D:\Belajar ML\training\label_2\000010.txt -> dataset/train/labels\000010.txt
Converted: D:\Belajar ML\trainin

In [6]:
import torch
print("DirectML available:", torch.cuda.is_available())
print("Available devices:", torch.cuda.device_count())


DirectML available: False
Available devices: 0
